# Lab 3: Extending Logistic Regression
## by Michael Doherty, Leilani Guzman, and Carson Pittman

## 1. Business Understanding

In 1633, the Japanese government, feeling threatened by Spanish and Portugese influence within their country, banned foreign goods as part of their isolationist foreign policy (called [Sakoku](https://en.wikipedia.org/wiki/Sakoku)). As part of this ban, European playing cards were forbidden, so Japanese playing cards, called [hanafuda](https://lammuseum.wfu.edu/2021/12/japanese-hanafuda-cards/), were developed; however, hanafuda struggled to gain popularity among the Japanese populace. Centuries later, on November 22, 1859 in Kyoto, Japan, a man named Fusajiro Yamauchi was born. An aspiring artist and entrepreneur, Yamauchi saw an opportunity with hanafuda, so in 1889 he opened a shop to sell his own handcrafted cards. What was the name of that shop? [Nintendo Koppai](https://www.lifewire.com/fusajiro-yamauchi-founder-of-nintendo-729584) (which was later shortened to just Nintendo).

While Fusajiro Yamauchi would never know it, his card company would eventually become one of the biggest video game companies in the world. Since the mid-twentieth century, advancements in technology have propelled video games to become a multi-billion dollar industry. Yet, not all video games are made equal; from sports games like the [FIFA series](https://en.wikipedia.org/wiki/FIFA_(video_game_series)) to puzzle games like [Tetris](https://en.wikipedia.org/wiki/Tetris), there are several genres that a video game can belong to.

The dataset we've selected, titled "Global Video Game Sales", was created to analyze how a video game's genre relates to the platform it was released on, its publisher, and its sales (both globally and in various regions). The dataset consists of 16,600 instances with 11 multivariate attributes, comprised of numeric and categorical data types.

Ultimately, the prediction task for this dataset is to determine what a video game's genre is. Who might be interested in this research? For starters, video game companies may be interested to see which video game genre has the best sales. They may also find that certain genres are more popular in some markets than others (such as puzzle games being popular in Japan and sports games being popular in Europe). This information can help inform their strategies on what types of games they should make and where they should focus their marketing efforts.

...**SAY SOMETHING ABOUT HOW THIS MODEL WOULD BE USED FOR OFFLINE ANALYSIS**

So what does a good prediction algorithm for this dataset look like? There are several factors that need to be considered:
- **Accuracy**:
- **False Positives and False Negatives**:
- **Finding Useful Trends**: (???) (Like finding clear trends, such as: Nintendo games sell best in Japan or the most profitable genre is Shooter games, etc.)

...**ADD MORE**

Link to the dataset: https://www.kaggle.com/datasets/thedevastator/global-video-game-sales

## 2. Data Preparation

In [4]:
import pandas as pd

df = pd.read_csv("data/vgsales.csv")

df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [5]:
df.info()
rows_with_null_year = df[df["Year"].isna()]
rows_with_null_year

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
179,180,Madden NFL 2004,PS2,NaN,Sports,Electronic Arts,4.26,0.26,0.01,0.71,5.23
377,378,FIFA Soccer 2004,PS2,NaN,Sports,Electronic Arts,0.59,2.36,0.04,0.51,3.49
431,432,LEGO Batman: The Videogame,Wii,NaN,Action,Warner Bros. Interactive Entertainment,1.86,1.02,0.00,0.29,3.17
470,471,wwe Smackdown vs. Raw 2006,PS2,NaN,Fighting,NaN,1.57,1.02,0.00,0.41,3.00
607,608,Space Invaders,2600,NaN,Shooter,Atari,2.36,0.14,0.00,0.03,2.53
...,...,...,...,...,...,...,...,...,...,...,...
16307,16310,Freaky Flyers,GC,NaN,Racing,Unknown,0.01,0.00,0.00,0.00,0.01
16327,16330,Inversion,PC,NaN,Shooter,Namco Bandai Games,0.01,0.00,0.00,0.00,0.01
16366,16369,Hakuouki: Shinsengumi Kitan,PS3,NaN,Adventure,Unknown,0.01,0.00,0.00,0.00,0.01
16427,16430,Virtua Quest,GC,NaN,Role-Playing,Unknown,0.01,0.00,0.00,0.00,0.01


A lot of the missing data can be found in the dataset... (Madden NFL 2004 has multiple entries for each platform it was released on, and the other entries say it came out in 2003

## 3. Modeling

In [6]:
import numpy as np
from scipy.special import expit

# Binary Logistic Regression using vectorized implementation

class BinaryLogisticRegression:
    # Adjust default values for iterations and C as needed
    def __init__(self, eta, iterations=20, C=0.001):
        self.eta = eta          # The learning rate
        self.iterations = iterations
        self.C = C
    def __str__(self):
        if(hasattr(self, 'weights_')):
            return "Binary Logistic Regression Object with coefficients:\n" + str(self.weights_)
        return 'Binary Logistic Regression Object is untrained'
    
    @staticmethod
    def _add_bias(X):
        # Add a column of ones to the right side of X
        return np.hstack((X, np.ones((X.shape[0],1))))
    
    @staticmethod
    def _sigmoid(theta):
        return expit(theta)
    
    def _get_gradient(self, X, y):
        y_diff = y - self.predict_proba(X, add_bias=False).ravel()
        gradient = np.mean(X * y_diff[:,np.newaxis], axis = 0)
        gradient = gradient.reshape(self.weights_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C
        return gradient
    
    def predict_proba(self, X, add_bias=True):
        X_bias = self._add_bias(X) if add_bias else X
        return self._sigmoid(X_bias @ self.weights_)
    
    def predict(self, X):
        return (self.predict_proba(X) > 0.5)
    
    def fit(self, X, y):
        X_bias = self._add_bias(X)
        num_samples, num_features = X_bias.shape
        self.weights_ = np.zeros((num_features,1))    # Creating the weight vector

        for _ in range(self.iterations):
            gradient = self._get_gradient(X_bias, y)
            self.weights_ += self.eta * gradient
    

In [7]:
from numpy import ma
from scipy.optimize import minimize_scalar

## Optimizing 1: Steepest Ascent/Line Search

class LineSearchLogisticRegression(BinaryLogisticRegression):
    def __init__(self, line_iterations=20, **kwds):
        self.line_iterations = line_iterations
        super().__init__(**kwds)

        @staticmethod
        def objective_function(eta, X, y, w, gradient, C):
            w_new = w - gradient*eta
            gradient_new = expit(X @ w_new)
            return -np.sum(ma.log(g[y==1]))-ma.sum(ma.log(1-gradient_new[y==0])) + C*sum(w_new**2)
        
        def fit(self, X, y):
            X_bias = self._add_bias(X)
            num_samples, num_features = X_bias.shape
            self.weights_ = np.zeros((num_features,1))

            for _ in range (self.iterations):
                gradient = -self._get_gradient(X_bias, y)
                opts = {'maxiter': self.line_iterations}
                res = minimize_scalar(self.objective_function,
                                      bounds=(0,self.eta*10),
                                      args=(X_bias, y, self.weights_, gradient, self.C),
                                      method='bounded',
                                      options=opts)
                
            eta = res.x
            self.weights_ -= eta*gradient

In [8]:
## Optimizing 2: Stoachastic Gradient Ascent

class StochasticLogisticRegression(BinaryLogisticRegression):
    def _get_gradient(self, X, y):
        # Adjust batch size as needed, calculates gradient according to this batch size
        batch_size = 16
        idxs = np.random.choice(len(y), batch_size)

        y_diff = y[idxs] - self.predict_proba(X[idxs], add_bias=False).ravel()
        gradient= np.mean(X[idxs] * y_diff[:,np.newaxis], axis = 0)

        gradient = gradient.reshape(self.weights_.shape)
        gradient[1:] += -2 * self.weights_[1:] * self.C

        # This might need to be negative?
        return gradient


In [9]:
from scipy.optimize import fmin_bfgs
## Optimizing 3: Quasi-Newton Method (BFGS)

class BFGSLogisticRegression(BinaryLogisticRegression):

    @staticmethod
    def objective_function(w, X, y, C):
        gradient = expit(X @ w)
        return -ma.sum(ma.log(g[y==1])-ma.sum(ma.log(1-g[y==0]))) + C*sum(w**2)
    
    @staticmethod
    def objective_gradient(w, X, y, C):
        gradient = expit(X @ w)
        y_diff = y-gradient
        gradient = np.mean(X * y_diff[:,np.newaxis], axis = 0)
        gradient = gradient.reshape(w.shape)
        gradient[1:] += -2 * w[1:] * C
        return -gradient
    
    def fit(self, X, y):
        X_bias = self._add_bias(X)
        num_samples, num_features = X_bias.shape

        self.weights_ = fmin_bfgs(self.objective_function,
                                  np.zeros((num_features,1)),
                                  fprime=self.objective_gradient,
                                  args=(X_bias, y, self.C),
                                  gtol=1e-03,
                                  maxiter=self.iterations,
                                  disp=False)
        self.weights_ = self.weights_.reshape((num_features,1))

In [10]:

#Regularized Logistic Regression using vectorized implementation
class MultiClassLogisticRegression:
    # Adjust default values for iterations and C as needed, temp default for solver is LineSearchLogisticRegression
    def __init__(self, eta, iterations=20, C=0.0, solver=LineSearchLogisticRegression):
        self.eta = eta
        self.iterations = iterations
        self.C = C
        self.solver = solver
        self.classifiers_ = []

    def __str__(self):
        if(hasattr(self, 'weights_')):
            return "Multiclass Logistic Regression Object with coefficients:\n" + str(self.weights_)
        return 'Multiclass Logistic Regression Object is untrained'
    
    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.unique_classes_ = np.unique(y)
        num_unique_classes = len(self.unique_classes_)
        self.classifiers_ = []

        for i, y_value in enumerate(self.unique_classes_):
            # One vs All
            y_binary = (y==y_value)
            blr = self.solver(self.eta, self.iterations, self.C)
            blr.fit(X, y_binary)
            self.classifiers_.append(blr)
        
        self.weights_ = np.hstack([x.weights_ for x in self.classifiers_]).T

    def predict_proba(self, X):
        probabilities = []
        for blr in self.classifiers_:
            probabilities.append(blr.predict_proba(X))
        return np.hstack(probabilities)
    
    def predict(self, X):
        return self.unique_classes_[np.argmax(self.predict_proba(X), axis=1)]

## 4. Deployment

## 5. Exceptional Work (rename to what we actually end up doing)